<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/PCA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PCA as Noise Filtering

PCA can also be used as a filtering approach for noisy data.<br>
The idea is this: any components with variance much larger than the effect of the noise should be relatively unaffected by the noise.
So if you reconstruct the data using just the largest subset of principal components, you should be preferentially keeping the signal and throwing out the noise.<br>

Let's see how this looks with the digits data.
First we will plot several of the input noise-free data:

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
print("done")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

Load the data: hand drawn digits<br>

The data consists of 8×8 pixel images, meaning that they are 64-dimensional (or features)<br>

We can use PCA to project them to a more manageable number of dimensions, say two:

In [ ]:
digits = load_digits()
digits.data.shape

In [ ]:
X = digits.images.reshape((len(digits.images), -1))
y = digits.target

# **Plot the digits**

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

**Create a model to classifiy the digits** 

In [ ]:
model_orig = DecisionTreeClassifier()
model_orig.fit(X, y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_orig, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# **Add some random noise to create a noisy dataset, and re-plot it:**

In [ ]:
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

**Train the model on the noisy data**
What happens to the accuracy?

In [ ]:
model_noise = DecisionTreeClassifier()
model_noise.fit(noisy, y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_noise, noisy, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

It's clear the images are noisy, and contain spurious pixels.<br>
Let's train a PCA on the noisy data, requesting that the projection preserve a percentage of the variance:

In [ ]:
#Assignment 1 - make changes here
pca = PCA(0.20).fit(noisy)
pca.n_components_

In this case 50% of the variance amounts to 12 principal components. Now we compute these components, and then use the inverse of the transform to reconstruct the filtered digits:

In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

This signal preserving/noise filtering property makes PCA a very useful feature selection routine—for example, rather than training a classifier on very high-dimensional data, you might instead train the classifier on the lower-dimensional representation, which will automatically serve to filter out random noise in the inputs.

Train the model on the filtered data<br>
Note the improved accuracy over the noisy data, but the accuracy is not as good as the clean data. 

In [ ]:
model_filtered = DecisionTreeClassifier()
model_filtered.fit(filtered, y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_filtered, filtered,y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# **Assignment 1**<br>
1. Change the threshold for the variance for the filtered model. <br>
2. What is the optimum threshold for variance to improve the accuracy of the filtered model?
What happens to the accuracy of the model when the number of principal components changes?

# **Assignment 2**<br>
Plot the PCA curve for the filtered data. 
What is the optimum number of principal components?

In [ ]:
pca = PCA().fit(filtered)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');